In [5]:
# Dữ liệu sau có dữ liệu mất mát, trống, sai kiểu, trùng lặp,.....
# order_id,customer_id,order_date,total_amount,payment_method,status
path_csv = "D:/Python/2024/test_hadoop/hoc_spark/data/orders_bai3.csv"


1. Đếm số lượng đơn hàng theo từng trạng thái (pending, processing, completed, cancelled) và hiển thị kết quả.
2. Tính tổng doanh thu từ tất cả các đơn hàng đã hoàn thành.
3. Tìm đơn hàng có tổng số tiền lớn nhất và nhỏ nhất và hiển thị chi tiết của chúng.
4. Tính trung bình số tiền của các đơn hàng theo từng phương thức thanh toán.
5. Tính độ trễ trong xử lý đơn hàng (số ngày giữa ngày đặt hàng và ngày hoàn thành) và tính trung bình trễ.
6. Chuyển định dạng ngày tháng sang định dạng theo ngày đầu tiên của tháng đó và hiển thị kết quả.
7. Tìm các cặp đơn hàng có cùng khách hàng (customer_id) và sắp xếp theo ngày đặt hàng tăng dần.
8. Loại bỏ các dòng trùng lặp hoàn toàn dựa trên tất cả các trường dữ liệu và hiển thị kết quả.
9. Tính tổng số tiền của đơn hàng theo từng trạng thái và phương thức thanh toán, sau đó sắp xếp theo số tiền giảm dần.
10. Tạo một cột mới "processedamount" là tổng số tiền đã xử lý từ tổng số tiền ban đầu (totalamount).
11. Lọc ra các đơn hàng có tổng số tiền lớn hơn mức trung bình và hiển thị kết quả.
12. Tính tổng số tiền từ các đơn hàng có trạng thái "pending" và phương thức thanh toán là "Cash".
13. Tách trường "order_date" thành các trường "year", "month" và "day" riêng lẻ và hiển thị kết quả.
14. Tìm các đơn hàng có số tiền chẵn và sắp xếp theo ngày đặt hàng giảm dần.
15. Chuyển định dạng ngày tháng sang Unix Timestamp và hiển thị kết quả.

### tao ung dung

In [2]:
from pyspark.sql import SparkSession
ss = SparkSession.builder \
    .appName("bai 3") \
    .master("local[*]") \
    .getOrCreate()

In [3]:
sc = ss.sparkContext

### load data + cache

In [9]:
orders_rdd = sc.textFile(path_csv)

In [10]:
orders_rdd.take(5)

['1,101,2022-06-15,150.5,Credit Card,pending',
 '2,102,2022-06-16,,Cash,processing',
 '3,103,2022-06-17,error,PayPal,completed',
 '4,104,2022-06-18,180.75,Credit Card,cancelled',
 '1,102,2022-06-19,300.25,,pending']

In [11]:
# distinct id don hang:
id_info_mapped_rdd = orders_rdd.map(lambda x: (x.split(",")[0], x)) # tach lay tuple (id, info don hang)
distinct_id = id_info_mapped_rdd.reduceByKey(lambda x, y: y) # reduce by key lay gia tri cuoi cung cua moi key sau khi reduce
distinct_id = distinct_id.map(lambda x: x[1]) # tach lay info don hang
distinct_id.cache()
distinct_id.take(10)

['1,102,2022-06-19,300.25,,pending',
 '4,104,2022-06-18,180.75,Credit Card,cancelled',
 '8,108,2022-06-21,220.3,,processing',
 '10,109,2022-06-23,175.0,Credit Card,pending',
 '12,101,2022-06-25,error,PayPal,completed',
 '14,113,2022-06-27,missing,Cash,processing',
 '16,115,2022-06-30,error,Credit Card,pending',
 '17,116,2022-07-01,290.0,Credit Card,processing',
 '19,118,2022-07-03,invalid,Credit Card,cancelled',
 '20,119,2022-07-04,195.2,Credit Card,pending']

### 1. Đếm số lượng đơn hàng theo từng trạng thái (pending, processing, completed, cancelled) và hiển thị kết quả.


In [12]:
# dem so luong don hang nen id don hang can bo id trung lap, su dung distinct_id(da bien doi o tren)
trang_thai_don_hang_mapped_rdd = distinct_id.map(lambda x: (x.split(",")[-1], 1))
so_luong_don_hang_theo_trang_thai_rdd = trang_thai_don_hang_mapped_rdd.reduceByKey(lambda x, y: x + y)
so_luong_don_hang_theo_trang_thai_rdd.collect()

[('pending', 12),
 ('cancelled', 10),
 ('processing', 11),
 ('completed', 11),
 ('error', 4)]

### 2. Tính tổng doanh thu từ tất cả các đơn hàng đã hoàn thành.


In [14]:
def trang_thai_vs_gia(line):
    line_split = line.split(",")
    return (line_split[-1], line_split[-3])

In [15]:
trang_thai_vs_gia_mapped_rdd = orders_rdd.map(trang_thai_vs_gia)

In [20]:
trang_thai_vs_gia_mapped_rdd.take(5)

[('pending', '150.5'),
 ('processing', ''),
 ('completed', 'error'),
 ('cancelled', '180.75'),
 ('pending', '300.25')]

In [24]:
def fix_du_lieu(line):
    status, amount = line
    try:
        float(amount)
        return (status, amount)
    except:
        return (status, 0)

In [25]:
trang_thai_vs_gia_mapped_rdd = trang_thai_vs_gia_mapped_rdd.map(fix_du_lieu)

In [27]:
trang_thai_vs_gia_mapped_rdd.take(10)

[('pending', '150.5'),
 ('processing', 0),
 ('completed', 0),
 ('cancelled', '180.75'),
 ('pending', '300.25'),
 ('completed', 0),
 ('error', 0),
 ('processing', '220.3'),
 ('error', '280.5'),
 ('pending', '175.0')]

In [35]:
trang_thai_vs_gia_reduce_rdd = trang_thai_vs_gia_mapped_rdd.reduceByKey(lambda x, y: float(x) + float(y))

In [36]:
trang_thai_vs_gia_reduce_rdd.take(5)

[('pending', 1987.6000000000001),
 ('cancelled', 777.4000000000001),
 ('processing', 1527.15),
 ('completed', 1618.95),
 ('error', 826.5)]

In [32]:
trang_thai_vs_gia_reduce_rdd.filter(lambda x: x[0] == "completed").collect()

[('completed', 1618.95)]

### 3. Tìm đơn hàng có tổng số tiền lớn nhất và nhỏ nhất và hiển thị chi tiết của chúng.


In [63]:
def fix_du_lieu_amount(line):
    order_id, amount = line
    try:
        return (order_id, float(amount))
    except:
        return line
    

def is_int_or_float(line):
    return isinstance(line[1], (int, float))



def order_id_vs_amount(line):
    line_split = line.split(",")
    return (line_split[0], line_split[-3])

In [44]:
order_id_vs_amount_mapped_rdd = distinct_id.map(order_id_vs_amount)

In [45]:
order_id_vs_amount_mapped_rdd.take(5)

[('1', '300.25'),
 ('4', '180.75'),
 ('8', '220.3'),
 ('10', '175.0'),
 ('12', 'error')]

In [65]:
orders_rdd_fix_amount = order_id_vs_amount_mapped_rdd.map(fix_du_lieu_amount)
orders_rdd_fix_amount = orders_rdd_fix_amount.filter(is_int_or_float)

In [66]:
orders_rdd_fix_amount.take(5)

[('1', 300.25), ('4', 180.75), ('8', 220.3), ('10', 175.0), ('17', 290.0)]

In [71]:
id_don_hang_amount_nho_nhat = orders_rdd_fix_amount.min(lambda x: x[1])[0]


In [72]:
id_don_hang_amount_lon_nhat =orders_rdd_fix_amount.max(lambda x: x[1])[0]

In [76]:
print("don hang co tong so tien nho nhat la: ", distinct_id.filter(lambda x: x.split(",")[0] == id_don_hang_amount_nho_nhat).collect()[0])
print("don hang co tong so tien lon nhat la: ", distinct_id.filter(lambda x: x.split(",")[0] == id_don_hang_amount_lon_nhat).collect())

don hang co tong so tien nho nhat la:  10,109,2022-06-23,175.0,Credit Card,pending
don hang co tong so tien lon nhat la:  ['26,,2022-07-10,340.2,Cash,completed']


### 5. Tính độ trễ trong xử lý đơn hàng (số ngày giữa ngày đặt hàng và ngày hoàn thành) và tính trung bình trễ.
